In [ ]:
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

import torch

model = T5ForConditionalGeneration.from_pretrained("castorini/monot5-3b-msmarco-10k", torch_dtype=torch.float16) #t5 3b
# model = T5ForConditionalGeneration.from_pretrained('castorini/monot5-base-msmarco-10k')

tokenizer = AutoTokenizer.from_pretrained("t5-3b")

device = torch.device("cuda")
model.to(device)

In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

reranker = MonoT5(model=model)

In [ ]:
import json

data = json.load(open("./data/strategyqa_torerank.json",'r'))

In [ ]:
from tqdm import tqdm

for item in tqdm(data):
    for d in item['decompositions']:
        texts = [Text(p['contents'], {'docid': i,"title":p,"contents":p['contents'],"id":p['id'],   }, 0) for i,p in enumerate(d['documents'])]
        query = Query(d['question'])
        reranked = reranker.rerank(query, texts)
        reranked.sort(key=lambda x: x.score, reverse=True)
        d['documents'] = [t.metadata for t in reranked]


In [ ]:
json.dump(data, open("data/strategyqa_reranked.json","w"))